# 05. Undirected Spark Exercise Solutions

### initialisation

In [1]:
import findspark
findspark.init()
import pyspark
sc=pyspark.SparkContext.getOrCreate()
sc.setLogLevel('ERROR')

#where the data is stored
datafiles='*.csv'

#initialisation of spark sql session
from pyspark.sql import Row, SparkSession
spark = SparkSession.builder.getOrCreate()

# read the wind data from CSV files
df = spark.read.csv(datafiles, header=True)

3 solutions:
1. pandas
2. MR
3. Spark SQL


### pandas

In [2]:
import pandas as pd
pddf=df.toPandas()

In [3]:
pddf.head()

,date,id,name,address1,address2,town,district,postcode,extra
0,201512,A81001,THE DENSHAM SURGERY,THE HEALTH CENTRE,LAWSON STREET,STOCKTON ON TEES,CLEVELAND,TS18 1HU,...
1,201512,A81002,QUEENS PARK MEDICAL CENTRE,QUEENS PARK MEDICAL CTR,FARRER STREET,STOCKTON ON TEES,CLEVELAND,TS18 2AW,...
2,201512,A81003,VICTORIA MEDICAL PRACTICE,THE HEALTH CENTRE,VICTORIA ROAD,HARTLEPOOL,CLEVELAND,TS26 8DB,...
3,201512,A81004,WOODLANDS ROAD SURGERY,6 WOODLANDS ROAD,,MIDDLESBROUGH,CLEVELAND,TS1 3BE,...
4,201512,A81005,SPRINGWOOD SURGERY,SPRINGWOOD SURGERY,RECTORY LANE,GUISBOROUGH,,TS14 7DJ,...


In [7]:
pddf['postcodeArea']=pddf['postcode'].map(lambda x: x.split()[0])

In [11]:
pddf.head()

,date,id,name,address1,address2,town,district,postcode,extra,postcodeArea
0,201512,A81001,THE DENSHAM SURGERY,THE HEALTH CENTRE,LAWSON STREET,STOCKTON ON TEES,CLEVELAND,TS18 1HU,...,TS18
1,201512,A81002,QUEENS PARK MEDICAL CENTRE,QUEENS PARK MEDICAL CTR,FARRER STREET,STOCKTON ON TEES,CLEVELAND,TS18 2AW,...,TS18
2,201512,A81003,VICTORIA MEDICAL PRACTICE,THE HEALTH CENTRE,VICTORIA ROAD,HARTLEPOOL,CLEVELAND,TS26 8DB,...,TS26
3,201512,A81004,WOODLANDS ROAD SURGERY,6 WOODLANDS ROAD,,MIDDLESBROUGH,CLEVELAND,TS1 3BE,...,TS1
4,201512,A81005,SPRINGWOOD SURGERY,SPRINGWOOD SURGERY,RECTORY LANE,GUISBOROUGH,,TS14 7DJ,...,TS14


In [18]:
bn1=pddf[pddf['postcodeArea']=="BN1"]
gu27=pddf[pddf['postcodeArea']=='GU27']

In [17]:
bn1.describe()

,date,id,name,address1,address2,town,district,postcode,extra,postcodeArea
count,24,24,24,24,24,24,24,24,24,24
unique,1,24,24,20,17,3,3,17,1,1
top,201512,G81042,SHIP STREET SURGERY,COUNTY OAK MEDICAL CENTRE,CARDEN HILL,BRIGHTON,EAST SUSSEX,BN1 8DD,...,BN1
freq,24,1,1,4,4,22,20,4,24,24


In [19]:
gu27.describe()

,date,id,name,address1,address2,town,district,postcode,extra,postcodeArea
count,1,1,1,1,1,1,1,1,1,1
unique,1,1,1,1,1,1,1,1,1,1
top,201512,H81062,HASLEMERE HEALTH CENTRE,HASLEMERE HEALTH CENTRE,CHURCH LANE,HASLEMERE,SURREY,GU27 2BQ,...,GU27
freq,1,1,1,1,1,1,1,1,1,1


## Answer
There are 24 doctors surgeries in BN1
There is 1 doctors surgery in GU27

In [28]:
pddf.groupby('postcodeArea').count()

,date,id,name,address1,address2,town,district,postcode,extra
postcodeArea,,,,,,,,,
AL1,3,3,3,3,3,3,3,3,3
AL10,5,5,5,5,5,5,5,5,5
AL2,3,3,3,3,3,3,3,3,3
AL3,8,8,8,8,8,8,8,8,8
AL5,4,4,4,4,4,4,4,4,4
...,...,...,...,...,...,...,...,...,...
YO60,1,1,1,1,1,1,1,1,1
YO61,4,4,4,4,4,4,4,4,4
YO62,4,4,4,4,4,4,4,4,4


## Map Reduce

In [34]:
practices=df.rdd
mapped= practices.map(lambda s:(s.id,s.postcode))
areas = mapped.map(lambda s:(s[0],s[1].split()[0]))

In [35]:
for key,postcode in areas.collect():
    print(key,postcode)

A81001 TS18
A81002 TS18
A81003 TS26
A81004 TS1
A81005 TS14
A81006 TS18
A81007 TS24
A81008 TS6
A81009 TS5
A81011 TS24
A81012 TS3
A81013 TS12
A81014 TS23
A81015 TS10
A81016 TS1
A81017 TS17
A81018 TS10
A81019 TS3
A81020 TS4
A81021 TS6
A81022 TS12
A81023 TS1
A81025 TS18
A81026 TS5
A81027 TS15
A81029 TS1
A81030 TS1
A81031 TS24
A81032 TS14
A81033 TS3
A81034 TS17
A81035 TS1
A81036 TS20
A81037 TS1
A81038 TS3
A81039 TS16
A81040 TS23
A81041 TS24
A81042 TS6
A81043 TS6
A81044 TS25
A81045 TS10
A81046 TS18
A81047 TS11
A81048 TS11
A81049 TS3
A81051 TS5
A81052 TS10
A81053 TS13
A81054 TS10
A81056 TS23
A81057 TS23
A81058 TS8
A81060 TS26
A81063 TS24
A81064 TS1
A81065 TS6
A81066 TS21
A81067 TS18
A81069 TS10
A81070 TS25
A81602 TS23
A81605 TS10
A81608 TS17
A81609 TS18
A81610 TS23
A81611 TS8
A81612 TS25
A81613 TS26
A81618 TS12
A81621 TS3
A81622 TS26
A81623 TS18
A82003 LA12
A82004 CA9
A82005 LA22
A82006 CA16
A82007 LA14
A82008 LA14
A82009 LA13
A82010 LA14
A82012 CA8
A82013 CA17
A82014 CA7
A82015 CA1
A82016 CA

C83052 LN1
C83053 NG33
C83054 PE10
C83055 PE22
C83056 LN11
C83057 PE20
C83058 LN4
C83059 PE21
C83060 PE21
C83061 DN36
C83062 LN4
C83063 PE12
C83064 LN12
C83065 PE12
C83067 NG23
C83068 PE9
C83071 LN6
C83072 LN2
C83073 LN1
C83074 LE15
C83075 NG31
C83078 LN5
C83079 LN2
C83080 NG31
C83605 LN9
C83611 LN5
C83613 LN7
C83614 PE20
C83615 LN1
C83617 PE6
C83626 LN1
C83628 NG32
C83631 PE11
C83633 DN21
C83634 LN10
C83635 LN10
C83637 LN6
C83641 LN1
C83643 LN8
C84001 S81
C84002 NG16
C84003 NG9
C84004 NG6
C84005 LE12
C84008 NG22
C84009 NG24
C84010 NG4
C84011 NG5
C84012 NG17
C84013 DN22
C84014 NG17
C84016 NG19
C84017 NG13
C84018 NG3
C84019 NG24
C84020 NG18
C84021 NG22
C84023 NG7
C84024 S80
C84025 NG13
C84026 NG5
C84028 NG11
C84029 NG24
C84030 NG9
C84031 NG18
C84032 NG16
C84033 NG4
C84034 NG6
C84035 DN22
C84036 NG19
C84037 NG21
C84039 NG7
C84042 NG9
C84043 NG6
C84044 NG8
C84045 NG23
C84046 NG11
C84047 NG14
C84048 NG12
C84049 NG25
C84051 NG18
C84052 DN10
C84053 NG15
C84055 NG5
C84057 NG19
C84059 NG19
C84

E84080 NW2
E84083 HA1
E84084 HA9
E84086 NW2
E84601 HA3
E84617 HA1
E84620 HA9
E84624 NW10
E84626 HA0
E84635 HA9
E84637 NW10
E84638 HA0
E84645 NW10
E84646 HA3
E84647 HA3
E84653 HA8
E84656 NW10
E84657 HA7
E84658 HA3
E84663 HA3
E84665 NW2
E84667 W9
E84669 HA0
E84674 NW2
E84676 HA2
E84678 HA9
E84680 HA1
E84681 HA1
E84684 HA9
E84685 HA0
E85001 TW7
E85003 W14
E85004 TW8
E85005 W12
E85006 UB2
E85007 TW1
E85008 SW6
E85012 UB1
E85013 W7
E85014 W13
E85015 TW3
E85016 W6
E85018 TW5
E85019 W3
E85020 W6
E85021 UB5
E85023 UB1
E85024 TW14
E85025 SW6
E85026 W13
E85028 W3
E85029 SW6
E85030 W4
E85032 W12
E85033 W6
E85034 W13
E85035 TW13
E85038 SW6
E85040 W4
E85041 W7
E85042 W12
E85044 TW3
E85045 TW13
E85046 UB6
E85048 W12
E85049 UB2
E85050 UB6
E85051 W7
E85052 TW5
E85053 UB5
E85054 UB6
E85055 W12
E85056 TW13
E85057 W5
E85058 TW3
E85059 TW3
E85060 TW3
E85061 UB2
E85062 TW3
E85064 UB5
E85066 W4
E85069 UB6
E85071 TW13
E85074 W6
E85075 W4
E85077 W12
E85083 UB1
E85088 W7
E85090 UB2
E85091 W5
E85096 UB1
E85098 

G82211 CT3
G82212 DA1
G82215 TN12
G82217 CT20
G82218 DA4
G82219 CT10
G82221 DA9
G82224 TN17
G82225 BR8
G82226 ME8
G82227 CT15
G82228 CT4
G82229 ME17
G82230 ME5
G82231 ME10
G82232 CT20
G82233 ME3
G82600 ME8
G82604 ME15
G82605 TN17
G82622 ME7
G82630 CT10
G82631 ME4
G82634 ME10
G82635 ME8
G82639 DA1
G82641 ME15
G82647 DA1
G82648 DA12
G82649 CT9
G82650 CT10
G82651 TN3
G82652 CT18
G82653 ME1
G82656 ME5
G82658 TN25
G82662 CT16
G82665 TN29
G82666 CT7
G82667 ME9
G82670 ME1
G82671 ME9
G82679 ME2
G82681 TN12
G82682 ME12
G82684 CT18
G82686 ME12
G82687 ME12
G82688 TN23
G82690 DA12
G82691 ME17
G82693 ME10
G82696 CT14
G82697 ME5
G82698 ME9
G82700 CT16
G82702 ME10
G82704 ME8
G82706 ME7
G82708 ME2
G82710 DA2
G82711 ME1
G82712 TN23
G82715 TN2
G82719 ME5
G82721 ME8
G82722 BR8
G82726 CT5
G82727 ME8
G82729 CT17
G82730 TN23
G82732 TN18
G82733 TN17
G82735 TN23
G82737 ME8
G82739 ME5
G82741 ME5
G82744 ME4
G82751 ME17
G82753 ME4
G82754 TN11
G82757 ME12
G82760 CT19
G82762 ME7
G82763 ME7
G82764 ME1
G83001 SE3
G8

K82004 HP6
K82005 SL8
K82006 SL0
K82007 MK18
K82008 SL9
K82009 MK11
K82010 HP12
K82011 HP9
K82012 HP15
K82013 MK2
K82014 HP20
K82015 MK3
K82016 MK16
K82017 HP11
K82018 HP19
K82019 HP21
K82020 HP13
K82021 HP27
K82022 HP13
K82023 SL7
K82024 HP5
K82025 MK15
K82026 MK3
K82027 MK14
K82028 HP17
K82029 HP10
K82030 HP11
K82031 SL2
K82032 MK14
K82033 SL1
K82034 HP27
K82035 HP16
K82036 HP11
K82037 HP5
K82038 HP20
K82039 MK2
K82040 HP19
K82042 HP22
K82043 MK18
K82044 HP13
K82045 SL2
K82046 HP9
K82047 HP18
K82048 HP14
K82049 HP14
K82050 MK2
K82051 SL9
K82053 HP13
K82054 MK6
K82055 UB9
K82057 MK46
K82058 HP5
K82059 MK2
K82060 MK14
K82061 LU7
K82062 MK2
K82064 MK6
K82065 MK13
K82066 HP10
K82067 MK8
K82068 HP18
K82069 MK18
K82070 LU7
K82073 HP22
K82074 MK16
K82076 MK5
K82078 SL9
K82603 HP12
K82610 MK6
K82615 MK7
K82617 MK11
K82618 HP16
K82621 HP7
K83002 NN17
K83003 NN2
K83004 NN13
K83005 NN8
K83006 NN15
K83007 NN10
K83008 NN2
K83009 NN3
K83010 NN4
K83011 NN8
K83012 NN1
K83013 NN16
K83014 NN2
K83015 N

M84050 CV21
M84051 CV11
M84055 CV10
M84059 CV31
M84060 B49
M84061 CV12
M84062 CV35
M84063 CV34
M84064 CV31
M84065 CV21
M84066 CV37
M84067 CV21
M84603 CV31
M84608 B80
M84609 CV12
M84612 CV9
M84615 CV10
M84616 CV23
M84617 CV37
M84618 CV12
M85001 B35
M85002 B20
M85003 B24
M85005 B10
M85006 B13
M85007 B31
M85008 B33
M85009 B20
M85010 B20
M85011 B25
M85013 B33
M85014 B24
M85015 B11
M85016 B44
M85018 B14
M85019 B42
M85020 B19
M85021 B13
M85023 B29
M85024 B10
M85025 B15
M85026 B76
M85027 B45
M85028 B17
M85029 B30
M85030 B31
M85031 B34
M85033 B72
M85034 B9
M85035 B32
M85037 B14
M85038 B75
M85041 B29
M85042 B29
M85043 B38
M85045 B14
M85046 B75
M85047 B31
M85048 B26
M85051 B12
M85052 B31
M85053 B7
M85055 B29
M85056 B29
M85058 B17
M85059 B14
M85060 B44
M85061 B9
M85062 B29
M85063 B24
M85064 B16
M85065 B24
M85066 B8
M85068 B72
M85069 B6
M85070 B23
M85071 B38
M85072 B21
M85074 B13
M85075 B9
M85076 B31
M85077 B30
M85078 B11
M85079 B35
M85081 B23
M85082 B21
M85083 B74
M85084 B12
M85085 B12
M85086 B31

P84047 M40
P84048 M22
P84049 M9
P84050 M18
P84051 M11
P84052 M12
P84053 M19
P84054 M40
P84055 M21
P84056 M20
P84059 M11
P84061 M23
P84062 M40
P84064 M4
P84065 M9
P84066 M21
P84067 M40
P84068 M21
P84070 M40
P84071 M21
P84072 M14
P84605 M9
P84611 M13
P84616 M14
P84623 M8
P84626 M14
P84630 M15
P84631 M8
P84635 M16
P84637 M11
P84639 M14
P84640 M8
P84644 M12
P84645 M8
P84650 M16
P84651 M23
P84652 M21
P84663 M9
P84665 M20
P84669 M15
P84672 M23
P84673 M4
P84676 M21
P84678 M20
P84679 M9
P85001 OL1
P85002 OL1
P85003 OL1
P85004 OL9
P85005 OL4
P85006 OL2
P85007 OL1
P85008 M35
P85010 OL9
P85011 OL9
P85012 OL8
P85013 OL2
P85014 OL4
P85015 OL4
P85016 OL3
P85017 OL2
P85018 OL9
P85019 OL2
P85020 OL4
P85021 OL4
P85022 OL4
P85025 OL9
P85026 M35
P85028 OL9
P85601 OL1
P85602 OL1
P85605 OL8
P85606 OL1
P85607 OL9
P85608 OL1
P85610 M35
P85612 OL8
P85613 OL9
P85614 OL2
P85615 OL9
P86001 OL16
P86002 OL11
P86003 OL11
P86004 M24
P86005 OL16
P86006 OL12
P86007 OL11
P86008 OL12
P86009 OL11
P86010 M24
P86011 OL10
P

B86678 BD11
N85645 CH49
C84717 NG6
P89618 OL5
P81785 PR1
C86626 DN5
F81751 CM3
D82106 NR2
A81630 TS5
N83638 WA10
G83680 SE9
G82812 CT11
C82676 LE5
P88636 SK4
K81663 RG4
E85750 TW7
P92651 WA3
D81638 CB21
M85803 B8
P81786 BB2
P81787 BB3
H81672 KT10
J82218 RG24
J83649 SN4
M84629 CV47
E82667 SG10
E84709 HA9
M83733 B77
P92652 M46
G82813 ME8
B85659 HD1
E87768 WC2R
P84696 M13
H82100 RH16
H83638 CR2
M85805 B26
C87621 S60
M87638 DY5
F81755 SS1
P91633 WA15
M81636 WR1
A83644 DH9
H85695 SW16
L81673 BS4
B83690 BD1
F85708 N8
J82674 SO30
P81788 BB11
C87622 S65
J81653 DT1
C82678 LE4
B83693 BD10
F84747 E14
M87639 B62
M87640 DY3
N85647 CH44
E86640 HA4
H84639 TW9
A81631 TS24
B81693 DN32
N82676 L12
A81632 TS18
M86638 CV2
E81639 LU1
P81791 FY4
N81655 CH1
P92653 WN3
F82686 RM6
C84720 NG7
F86731 E12
E85755 UB2
D82641 NR2
K84082 OX5
A83645 DH8
H83639 CR7
N82678 L9
F81756 SS14
F85710 EN4
J82220 RG24
J82685 SO14
P87664 M6
P87665 M6
H85697 SW17
F81757 CO11
D81645 PE3
B86682 LS10
P81793 BB2
E81077 MK45
M83738 WS1

Y02795 M24
Y02796 CM2
Y02801 CM16
Y02804 NN8
Y02805 SN1
Y02806 SO41
Y02807 RM17
Y02808 MK6
Y02809 CT6
Y02811 BR3
Y02812 UB3
Y02815 S72
Y02816 BN21
Y02822 CH2
Y02823 E15
Y02824 IG1
Y02826 DA11
Y02827 OL8
Y02833 S80
Y02835 B60
Y02836 OL11
Y02837 FY7
Y02838 SO16
Y02842 W10
Y02845 B91
Y02846 NG1
Y02847 NG2
Y02848 TW15
Y02849 M1
Y02851 BN1
Y02854 FY3
Y02859 KT12
Y02860 KT12
Y02861 GU22
Y02862 GU22
Y02863 GU22
Y02865 DN2
Y02867 ST6
Y02868 ST1
Y02869 W10
Y02870 CW1
Y02873 BS2
Y02875 OL1
Y02879 CV7
Y02881 ST4
Y02883 BR1
Y02884 TS8
Y02885 WN5
Y02886 WN7
Y02887 WN7
Y02888 PE13
Y02889 B8
Y02890 M14
Y02893 B33
Y02896 HU1
Y02897 SE3
Y02898 BD20
Y02900 MK10
Y02901 WS1
Y02902 HR2
Y02906 W12
Y02907 DA7
Y02908 HP6
Y02909 HP2
Y02910 E2
Y02914 N1
Y02916 HA1
Y02917 CM17
Y02927 ST6
Y02928 E16
Y02931 B8
Y02933 OL8
Y02935 CA14
Y02936 SK15
Y02937 SL6
Y02943 BL2
Y02945 NG8
Y02946 SW11
Y02947 N9
Y02952 TN15
Y02953 ME20
Y02954 RM16
Y02957 SE8
Y02960 M12
Y02961 B18
Y02962 CR9
Y02963 B19
Y02967 SW15
Y02968 CR4
Y02

Y04592 MK40
Y04593 L8
Y04594 CO4
Y04596 LA14
Y04597 BS15
Y04599 TS24
Y04600 BL1
Y04601 SM5
Y04602 CB2
Y04603 CR0
Y04604 CR8
Y04605 M20
Y04606 OL4
Y04607 AL7
Y04608 RG41
Y04609 ME8
Y04610 HD5
Y04611 S40
Y04612 DE7
Y04613 DE1
Y04614 SN25
Y04615 ME8
Y04616 ME8
Y04618 ME8
Y04621 HU5
Y04622 MK6
Y04623 N21
Y04625 CM16
Y04626 DE11
Y04627 DE5
Y04628 EX16
Y04630 BS37
Y04631 BA2
Y04632 S60
Y04633 MK11
Y04634 GU14
Y04635 L35
Y04636 ME7
Y04637 NG21
Y04638 B97
Y04639 DY10
Y04640 HP23
Y04641 WR5
Y04645 DA8
Y04647 E16
Y04649 HP6
Y04651 WA12
Y04654 HR1
Y04655 OL16
Y04656 B13
Y04657 BB4
Y04658 HA9
Y04659 SE14
Y04660 S72
Y04661 SS0
Y04662 EX5
Y04663 B4
Y04664 CH64
Y04665 OL16
Y04666 RH10
Y04667 MK11
Y04668 GL51
Y04669 DN14
Y04670 NR2
Y04671 DA8
Y04672 SO14
Y04673 WR1
Y04675 B16
Y04676 GL7
Y04678 RH12
Y04679 CT8
Y04680 WF4
Y04681 WD18
Y04682 CV1
Y04683 PL4
Y04684 RM1
Y04685 W10
Y04687 DH4
Y04688 SR5
Y04689 NE38
Y04690 SE15
Y04691 NG24
Y04692 OL1
Y04694 PL25
Y04695 HP6
Y04696 OL16
Y04697 DY8
Y04698 CV1
Y0

map each postcode area to a 1

In [58]:
counts=areas.map(lambda s:(s[1],1))

reduce by key to add up the counts for each postcode area

In [59]:
reduced=counts.reduceByKey(lambda a,b:a+b)

In [60]:
interested=["BN1","GU27"]
for (k,v) in reduced.collect():
    if k in interested:
        print(k,v)
        

BN1 24
GU27 1


## Spark SQL


In [29]:
df.registerTempTable('practices')

In [30]:
spark.sql("SELECT postcode, count(id) as count from practices group by postcode").show()

+--------------------+-----+
|            postcode|count|
+--------------------+-----+
|TS18 1YE         ...|    1|
|SR7 9AB          ...|    1|
|DL2 1AA          ...|    1|
|NE3 1NB          ...|    1|
|NE3 5JP          ...|    1|
|NE29 0SF         ...|    2|
|HU9 5AD          ...|    1|
|DN34 5DA         ...|    1|
|DE24 8QJ         ...|    1|
|LE4 1EF          ...|    1|
|NG3 3GG          ...|    3|
|DN1 2DS          ...|    1|
|S6 4FA           ...|    1|
|S3 9DA           ...|    1|
|IP7 7EX          ...|    1|
|IP12 3DA         ...|    1|
|SW6 6JD          ...|    1|
|RM15 5LP         ...|    1|
|SS8 0JH          ...|    1|
|CT21 5JY         ...|    1|
+--------------------+-----+
only showing top 20 rows



Now lets try to do it with the added postcode area field (from the RDD)

In [39]:
df2=areas.toDF(['id','postcode'])

In [44]:
df2.registerTempTable("areas")
areacounts=spark.sql("SELECT postcode, count(id) as count from areas group by postcode")

In [46]:
areacounts.show()

+--------+-----+
|postcode|count|
+--------+-----+
|     WF4|    4|
|     WF8|    3|
|    NR19|    3|
|     NR8|    2|
|    IP15|    1|
|    CT17|    2|
|    SE17|    6|
|    RG24|    5|
|    MK11|    4|
|    TQ14|    4|
|     L19|    4|
|     M19|    7|
|    RM19|    2|
|     DL6|    3|
|     CA5|    1|
|    YO22|    1|
|    DE21|    5|
|     LU5|    4|
|     TN1|    3|
|    KT14|    3|
+--------+-----+
only showing top 20 rows



So how do we get the postcodes we actually want?

In [53]:
areacounts.registerTempTable("acounts")


In [55]:
spark.sql("SELECT postcode,count from acounts WHERE postcode='CA5'").show()

+--------+-----+
|postcode|count|
+--------+-----+
|     CA5|    1|
+--------+-----+



In [56]:
spark.sql("SELECT postcode,count from acounts WHERE postcode='BN1'").show()

+--------+-----+
|postcode|count|
+--------+-----+
|     BN1|   24|
+--------+-----+



In [57]:
spark.sql("SELECT postcode,count from acounts WHERE postcode='GU27'").show()

+--------+-----+
|postcode|count|
+--------+-----+
|    GU27|    1|
+--------+-----+



Complete spark sql solution?

In [61]:
df

DataFrame[date: string, id: string, name: string, address1: string, address2: string, town: string, district: string, postcode: string, extra: string]

In [64]:
df['postcodeArea']=df['postcode'].map(lambda x: x.split()[0])

TypeError: 'Column' object is not callable